In [8]:
import anywidget
import traitlets
from traitlets import List, Any
from pathlib import Path
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
import scipy

grids = 2
boxs = 5

voxelarray = np.zeros((boxs * grids, boxs * grids, boxs * grids))

i = 1
for xi in range(0, 2):
    for yi in range(0, 2):
        for zi in range(0, 2):
            voxelarray[
                xi * boxs : xi * boxs + boxs,
                yi * boxs : yi * boxs + boxs,
                zi * boxs : zi * boxs + boxs,
            ] = i
            i += 1

voxelarray = np.uint8(voxelarray * 255 / i)

class HelloWidget(anywidget.AnyWidget):
    _esm = Path("widget.js")

    count = traitlets.Int(0).tag(sync=True)
    voxel_data = List(List(List(Any()))).tag(sync=True)




# Convert the NumPy array to a nested list

voxelarray = scipy.ndimage.gaussian_filter(voxelarray, sigma=0.4)

voxel_data_list = voxelarray.tolist()


counter = HelloWidget()


counter.voxel_data = voxel_data_list

counter

HelloWidget(voxel_data=[[[28, 28, 28, 28, 29, 54, 55, 56, 56, 56], [28, 28, 28, 28, 29, 54, 55, 56, 56, 56], […

In [15]:
import numpy as np
import scipy.ndimage
import ipywidgets as widgets
from IPython.display import display

sigma_slider = widgets.FloatSlider(value=0.1, min=0, max=4, step=0.01, description='Sigma:')

def apply_gaussian_filter(sigma):
    voxelarrayX = scipy.ndimage.gaussian_filter(voxelarray, sigma=sigma)
    voxel_data_list = voxelarrayX.tolist()
    counter.voxel_data = voxel_data_list
    print(f"Applied Gaussian filter with sigma: {sigma}")

# Display the slider
display(sigma_slider)

# Update function to be called when the slider value changes
def on_slider_change(change):
    apply_gaussian_filter(change['new'])

# Attach the update function to the slider
sigma_slider.observe(on_slider_change, names='value')

FloatSlider(value=0.1, description='Sigma:', max=4.0, step=0.01)